In [1]:
#spark
__author__ = 'vcoder'

import os
import sys

#change them when spark and pyspark path changes

#SPARK_HOME = "/home/vcoder/EDA/spark-1.5.0"
#SPARK_HOME_PYTHON = SPARK_HOME + "/python"

SPARK_HOME = "/home/worker/software/spark-1.5.0"
SPARK_HOME_PYTHON = SPARK_HOME + "/python"

os.environ['SPARK_HOME'] = SPARK_HOME
sys.path.append(SPARK_HOME_PYTHON)

from pyspark import SparkContext
from pyspark import SparkConf

import geoip2.database
reader = geoip2.database.Reader('GeoLite2-City.mmdb')

def ip2city(ip):
    try:
        city = reader.city(ip).city.name
    except:
        city = 'not found'
    return city

def ip2la(ip):
    try:
        la = reader.city(ip).location.latitude
    except:
        la = 0.
    return la

def ip2lo(ip):
    try:
        lo = reader.city(ip).location.longitude
    except:
        lo = 0.
    return lo


In [2]:
sc = SparkContext('local', 'ip2geo')

In [3]:
normalPath = 'csv/output.csv'
attackPath = 'csv/output.csv'

In [4]:
normalRaw = sc.textFile(normalPath).map(lambda x: x.split(',')) #delimiter may need to change
#print normalRaw.take(10)
attackRaw = sc.textFile(attackPath).map(lambda x: x.split(','))
#print attackRaw.take(10)
normalMax = normalRaw.map(lambda x: float(x[0])).max()
# print normalMax
normalMin = normalRaw.map(lambda x: float(x[0])).min()
# print normalMin
attackMax = attackRaw.map(lambda x: float(x[0])).max()
attackMin = attackRaw.map(lambda x: float(x[0])).min()

normalIp = normalRaw.map(lambda x: [x[1], x[2], float(x[0])*100/(normalMax - normalMin)])
attackIp = attackRaw.map(lambda x: [x[1], x[2], float(x[0])*100/(attackMax - attackMin)])
# print normalIp.take(10)
normalSrc = normalRaw.map(lambda x: x[1])
normalDst = normalRaw.map(lambda x: x[2])
attackSrc = attackRaw.map(lambda x: x[1])
attackDst = attackRaw.map(lambda x: x[2])
#print normalSrc.take(10)
allIPs = normalSrc.union(normalDst).union(attackSrc).union(attackDst).distinct().collect()
#print allIPs[0:9]

In [10]:
geoData = []
for ip in allIPs:
    lo = ip2lo(ip)
    la = ip2la(ip)
    geoData.append({'ip' : ip,
                    'longitude' : lo,
                    'lantitude' : la})

In [11]:
attackNum = 1000
normalNum = 1000
victimNum = 1000

attackDS = []
for x in attackIp.take(attackNum):
    src = x[0]
    dst = x[1]
    val = x[2]
    attackDS .append({'source': src,
                      'destination' : dst,
                      'value' : val})
    
normalDS = []
for x in normalIp.take(normalNum):
    src = x[0]
    dst = x[1]
    val = x[2]
    normalDS .append({'source': src,
                      'destination' : dst,
                      'value' : val})

In [12]:
victimData = []
victimIPs = attackIp.takeOrdered(victimNum, key = lambda x: -x[2])
for x in victimIPs:
    dst = x[1]
    val = x[2]
    victimData.append({'destination' : dst,
                       'value' :val})

In [14]:
import sys
ipDistribution = {'recordID': 1,#sys.argv[1], #add arg!!!!!!!!
                  'geoData' : geoData,
                  'attackDS' : attackDS,
                  'normalDS' : normalDS,
                  'victimData' : victimData}

In [17]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

dbName = 'test'
collectionName = 'maps'

db = client[dbName]
collection = db[collectionName]

collection.insert_one(ipDistribution)

In [16]:
sc.stop()